In [1]:
import pandas as pd
from taxbrain import TaxBrain, differences_plot, distribution_plot

In [2]:
puf = pd.read_csv("~/puf.csv")

In [3]:
# Use the Medicare payroll tax since uncapped.
FICA_MC_BASE = 0.029
YEAR = 2022
UBI_REFORM = {"UBI_21": 6000, "UBI_1820": 6000, "UBI_ecrt": 1}
reform_1pct = UBI_REFORM.copy()
reform_1pct["FICA_mc_trt"] = FICA_MC_BASE + 0.01
tb_1pct = TaxBrain(YEAR, YEAR, microdata=puf, reform=reform_1pct)
tb_1pct.run()

In [4]:
def rev(tb):
    return tb.weighted_totals("combined").loc["Difference"]

rev_per_1pct = rev(tb_1pct)
rev_per_1pct / 1e9

2022    95.018251
Name: Difference, dtype: float64

In [5]:
UBI = 6000
ubi_cost = (tb_1pct.microdata[["n1820", "n21"]].sum(axis=1) * tb_1pct.microdata.s006 / 100).sum() * UBI
ubi_cost / 1e9

1434.96514062

In [6]:
needed_payroll_tax = (ubi_cost / rev_per_1pct).round(1)
needed_payroll_tax

2022    15.1
Name: Difference, dtype: float64

In [7]:
# Budget-neutral
reform_bn_static = UBI_REFORM.copy()
reform_bn_static["FICA_mc_trt"] = FICA_MC_BASE + (needed_payroll_tax / 100)
tb_bn_static = TaxBrain(YEAR, YEAR, microdata=puf, reform=reform_bn_static)
tb_bn_static.run()

In [8]:
# Should be close to 1.
rev(tb_bn_static) / ubi_cost

2022    0.996812
Name: Difference, dtype: float64

In [9]:
# Budget-neutral with behavioral responses.
# Based on CBO (2012).
BEHAVIOR = {"sub": 0.25, "inc": -0.05}
reform_bn_beh = UBI_REFORM.copy()
reform_bn_beh["FICA_mc_trt"] = FICA_MC_BASE + 0.171
tb_bn_beh = TaxBrain(YEAR, YEAR, microdata=puf, reform=reform_bn_beh,
                     behavior=BEHAVIOR)
tb_bn_beh.run()

In [10]:
# Should be close to 1.
rev(tb_bn_beh) / ubi_cost

2022    0.998973
Name: Difference, dtype: float64